## Import library 

In [1]:
import numpy as np

## Read data and split to train data and test data

In [2]:
def read_data(path):
    with open(path) as f:
        content = f.readlines()
    content = [x.strip() for x in content]

    data = [[float(y) for y in x.split(' ')] for x in content if x[0] != "I"]

    # remove ID attribute out of data
    data = [ele[1:] for ele in data]

    train_data = data[:80]
    train_label = [ele.pop() for ele in train_data]

    test_data = data[80:]
    test_label = [ele.pop() for ele in test_data]

    train_data = np.asarray(train_data)
    train_label = np.asarray(train_label)
    test_data = np.asarray(test_data)
    test_label = np.asarray(test_label)

    return train_data, train_label, test_data, test_label

path ="./vidu4_lin_reg.txt"
train_data, train_label, test_data, test_label = read_data(path)

print("\n test-data : ",test_data)
print("\n test-label : ",test_label)
print("\n train-data : ",train_data)
print("\n train-label",train_label)



 test-data :  [[ 49.    24.   140.     4.3    5.5 ]
 [ 36.    23.   140.     4.3    4.2 ]
 [ 74.    21.   140.    17.     3.3 ]
 [ 53.    21.   140.     5.6    5.9 ]
 [ 56.    19.   140.     4.1    4.73]
 [ 60.    20.   160.     4.9    3.  ]
 [ 83.    21.   120.     7.9    5.88]
 [ 68.    23.   130.     4.     5.39]
 [ 69.    19.   100.     4.4    6.15]
 [ 31.    21.   120.     4.1    3.94]
 [ 34.    21.   140.     6.7    3.83]
 [ 41.    20.   120.     2.7    4.93]
 [ 72.    22.   160.     6.4    7.  ]
 [ 54.    22.   170.     6.2    8.18]
 [ 54.    28.   150.     4.2    8.16]
 [ 55.    24.   160.     5.     7.2 ]
 [ 76.    15.   140.     3.1    5.24]
 [ 70.    25.   180.     4.     4.4 ]
 [ 85.    21.   160.     5.2    5.2 ]
 [ 87.    22.   130.     9.     5.2 ]]

 test-label :  [0.8  0.7  1.   0.8  0.89 0.6  1.5  0.7  1.1  0.81 0.7  0.71 2.7  1.13
 1.7  0.9  1.16 1.   0.97 2.3 ]

 train-data :  [[ 56.    21.   160.    14.     6.  ]
 [ 76.    18.   150.    12.     4.97]
 [ 63.    16.

## QR

In [3]:
def qr_householder(A):
    # Compute QR decomposition of A using Householder reflection
    M = A.shape[0]
    N = A.shape[1]

    # set Q to the identity matrix
    Q = np.identity(M)

    # set R to zero matrix
    R = np.copy(A)

    for n in range(N):
        # vector to transform
        x = A[n:, n]
        k = x.shape[0]

        # compute ro=-sign(x0)||x||
        ro = -np.sign(x[0]) * np.linalg.norm(x)

        # compute the householder vector v
        e = np.zeros(k)
        e[0] = 1
        v = (1 / (x[0] - ro)) * (x - (ro * e))

    # apply v to each column of A to find R
    for i in range(N):
        R[n:, i] = R[n:, i] - (2 / (v@v)) * ((np.outer(v, v)) @ R[n:, i])

    # apply v to each column of Q
    for i in range(M):
        Q[n:, i] = Q[n:, i] - (2 / (v@v)) * ((np.outer(v, v)) @ Q[n:, i])

    return Q.transpose(), R

## Linear regression

In [4]:
def linear_regression(train_data, train_label):
    x_bars = np.concatenate((np.ones((train_data.shape[0], 1)), train_data), axis=1)

    Q, R = qr_householder(x_bars) # QR decomposition
    R_pinv = np.linalg.pinv(R) # calculate inverse matrix of R
    A = np.dot(R_pinv, Q.T) # apply formula

    return np.dot(A, train_label)

## Run program

In [5]:
w = linear_regression(train_data, train_label) # get result
w = w.T.tolist()
print("Regression coef:\n")
line = ['Intercept', 'Age', 'Cholesterol', 'Glucose', 'HA', 'BMI']
res = list(zip(line, w))
for o in res:
    print("{: >20}: {: >10}".format(*o))

Regression coef:

           Intercept: 0.04306436410330214
                 Age: 0.00898919688929678
         Cholesterol: -0.0004774242218528804
             Glucose: 0.0026021798675557707
                  HA: 0.008086342231978107
                 BMI: 0.007085352341923775


## Run program with test data

In [11]:
y_pre = [w[0]]*len(test_data)
for val_index in range(0, len(test_data)):
    for i in range(0,len(test_data[val_index])):
        y_pre[val_index] += w[i+1]*test_data[val_index][i]

error = np.abs(test_label - y_pre)
mean = np.mean(error)
variance = np.var(error)

print("validate predict:\n",y_pre)
print("\nMean of error is: ", mean)
print("Variance of error is: ", variance)

validate predict:
 [0.9101227212902697, 0.7845296279067635, 1.2233936873821376, 0.9608581673513565, 0.9683612308749348, 1.0500956057954387, 1.1969473567658557, 1.0521677963059988, 0.9936206987286872, 0.6850354344975742, 0.7842917235621115, 0.7710984473061301, 1.1974820427389565, 1.0684417447242467, 0.9972192105312192, 1.033807038520821, 1.1455820530106848, 1.1922858362001707, 1.292361781627831, 1.2625254556394154]

Mean of error is:  0.30532598337436617
Variance of error is:  0.1341770355526352
